In [1]:
from pyspark.sql.functions import col, monotonically_increasing_id, year, month, dayofmonth, dayofweek, to_date

# -------------------------------
# Read Silver Layer Data
# -------------------------------
silver_df = spark.read.format("delta").load("Tables/silver_liquor_sales")

StatementMeta(, da49a8d6-362a-4df0-a3d8-7d12e088b1c4, 3, Finished, Available, Finished)

**Date Dimension Creation**

In [2]:
dim_date = (silver_df
    .select(col("Date").alias("InvoiceDate"))
    .dropDuplicates()
    .withColumn("DateKey", monotonically_increasing_id())
    .withColumn("Year", year(col("InvoiceDate")))
    .withColumn("Month", month(col("InvoiceDate")))
    .withColumn("Day", dayofmonth(col("InvoiceDate")))
    .withColumn("Weekday", dayofweek(col("InvoiceDate")))
)

dim_date.write.format("delta").mode("overwrite").save("Tables/gold_dim_date")

StatementMeta(, da49a8d6-362a-4df0-a3d8-7d12e088b1c4, 4, Finished, Available, Finished)

**Store Dimension Creation**

In [4]:
dim_store = (silver_df
    .select(
        col("Store_Number").alias("StoreKey"),
        col("Store_Name").alias("StoreName"),
        "Address", "City", "Zip_Code", "County_Number", "County"
    )
    .dropDuplicates(["StoreKey"])
)


dim_store.write.format("delta").mode("overwrite").save("Tables/gold_dim_store")

StatementMeta(, da49a8d6-362a-4df0-a3d8-7d12e088b1c4, 6, Finished, Available, Finished)

**Vendor Dimension Creation**

In [5]:
dim_vendor = (silver_df
    .select(
        col("Vendor_Number").alias("VendorKey"),
        col("Vendor_Name").alias("VendorName")
    )
    .dropDuplicates(["VendorKey"]) )

dim_vendor.write.format("delta").mode("overwrite").save("Tables/gold_dim_vendor")

StatementMeta(, da49a8d6-362a-4df0-a3d8-7d12e088b1c4, 7, Finished, Available, Finished)

**Product Dimension Creation**

In [6]:
dim_product = (silver_df
    .select(
        col("Item_Number").alias("ProductKey"),
        col("Item_Description").alias("ProductName"),
        col("Category").alias("CategoryKey"),
        col("Category_Name").alias("CategoryName"),
        "Pack","Bottle_Volume__ml_"
    )
    .dropDuplicates(["ProductKey"]))

dim_product.write.format("delta").mode("overwrite").save("Tables/gold_dim_product")

StatementMeta(, da49a8d6-362a-4df0-a3d8-7d12e088b1c4, 8, Finished, Available, Finished)

**Fact Table Creation**

In [7]:
# Join surrogate keys from dimensions
fact_sales = (silver_df
    .join(dim_date, silver_df["Date"] == dim_date["InvoiceDate"], "left")
    .join(dim_store, silver_df["Store_Number"] == dim_store["StoreKey"], "left")
    .join(dim_vendor, silver_df["Vendor_Number"] == dim_vendor["VendorKey"], "left")
    .join(dim_product, silver_df["Item_Number"] == dim_product["ProductKey"], "left")
    .select(
        col("Invoice_Item_Number").alias("InvoiceKey"),
        col("DateKey"),
        col("StoreKey"),
        col("VendorKey"),
        col("ProductKey"),
        col("Bottles_Sold"),
        col("State_Bottle_Cost"),
        col("State_Bottle_Retail"),
        col("Sale__Dollars_").alias("SaleDollars"),
        col("Volume_Sold__Liters_").alias("VolumeLiters"),
        col("Volume_Sold__Gallons_").alias("VolumeGallons")
    )
)

fact_sales.write.format("delta").mode("overwrite").save("Tables/gold_fact_liquor_sales")

StatementMeta(, da49a8d6-362a-4df0-a3d8-7d12e088b1c4, 9, Finished, Available, Finished)